In [50]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn

In [131]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(url)
df

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


In [132]:
X = df.drop('Iris-setosa', axis=1)
Y = df['Iris-setosa']

In [133]:
X.head()

,5.1,3.5,1.4,0.2
0,4.9,3.0,1.4,0.2
1,4.7,3.2,1.3,0.2
2,4.6,3.1,1.5,0.2
3,5.0,3.6,1.4,0.2
4,5.4,3.9,1.7,0.4


In [134]:
Y

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
144    Iris-virginica
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
Name: Iris-setosa, Length: 149, dtype: object

In [135]:
type(X), type(Y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [136]:
X = X.values
Y = Y.values

In [137]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [138]:
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.transform(Y_test)
Y_test

array([0, 1, 0, 0, 2, 1, 1, 1, 0, 2, 2, 0, 2, 2, 0, 1, 2, 2, 1, 2, 0, 0,
       1, 0, 0, 2, 0, 2, 0, 1])

In [139]:
type(X_train), type(Y_train)

(numpy.ndarray, numpy.ndarray)

In [140]:
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train)

X_test = torch.from_numpy(X_test)
Y_test = torch.from_numpy(Y_test)

In [141]:
type(X_train), type(Y_train)

(torch.Tensor, torch.Tensor)

In [148]:
class SingleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 3, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(3, dtype=torch.float64, requires_grad=True)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        return z

    def loss_func(self, y_pred, y):
        return self.loss(y_pred, y)

In [143]:
learning_rate = 0.1
epochs = 100

In [149]:
model = SingleNN(X_train)

for i in range(epochs):
    
    y_pred = model.forward(X_train)    

    loss = model.loss_func(y_pred, Y_train)

    loss.backward()

    with torch.no_grad():
        # model.weights.sub_(learning_rate*model.weights.grad)
        model.weights -= learning_rate* model.weights.grad

        model.bias -= learning_rate* model.bias.grad

    model.weights.grad.zero_()
    model.bias.grad.zero_()

    if (i % 10 == 0):
        print(f'Epochs {i}\t Loss: {loss.item()}')

Epochs 0	 Loss: 2.1801362250025975
Epochs 10	 Loss: 0.8788372266396681
Epochs 20	 Loss: 0.8115840694700518
Epochs 30	 Loss: 0.7634241654234538
Epochs 40	 Loss: 0.7232420281078425
Epochs 50	 Loss: 0.6868834280823994
Epochs 60	 Loss: 0.6524997347044769
Epochs 70	 Loss: 0.6191811612621025
Epochs 80	 Loss: 0.5864626873787945
Epochs 90	 Loss: 0.55411689322415


In [164]:
labels = ['Setosa', 'Versicolor', 'Virginica']

In [204]:
## model evaluation
with torch.no_grad():
    print(f'Idx\tPredicted Val\t\tActual Value')
    print()
    for i, data in enumerate(X_test):
        y_pred = model.forward(data)
        y_pred = y_pred.argmax().item()
        
        print(f'{i+1}.)\t{y_pred}\t\t\t{Y_test[i]}')

Idx	Predicted Val		Actual Value

1.)	0			0
2.)	2			1
3.)	0			0
4.)	0			0
5.)	2			2
6.)	2			1
7.)	2			1
8.)	2			1
9.)	0			0
10.)	2			2
11.)	2			2
12.)	0			0
13.)	2			2
14.)	2			2
15.)	0			0
16.)	2			1
17.)	2			2
18.)	2			2
19.)	2			1
20.)	2			2
21.)	0			0
22.)	0			0
23.)	2			1
24.)	0			0
25.)	0			0
26.)	2			2
27.)	0			0
28.)	2			2
29.)	0			0
30.)	2			1


In [213]:
with torch.no_grad():
    y_pred_logits = model.forward(X_test)

    y_pred = torch.argmax(y_pred_logits, dim=1)

    correct_pred = (y_pred == Y_test).sum().item()
    total = Y_test.size(0)

    accuracy = (correct_pred/ total)* 100
    print(f'Accuracy on Single Neuron\t{accuracy:.2f}%')

Accuracy on Single Neuron	73.33%
